# GitHub pipelines

GitHub pipelines is githubs answer to CI/CD (continuous integration and continuous deployment). In this lecture we will first look into a few examples, and then learn how to build an automated pipeline.

What it is typically used for:
* Running Tests: Automatically test code every time a commit is pushed or a pull request is made.
* Building and Deploying Code: Build your applications and deploy them to cloud services or servers.
* Code Linting and Formatting: Automatically check for code style and formatting issues.
* Automation of Routine Tasks: Trigger workflows for various tasks like generating reports, managing issues, or even interacting with other services via APIs.

A configuration file states what actions should be performed in the pipeline. In a GitHub repository, the file is located under .github/workflows/, with a .yml (YAML)-format. The standard is to let pipelines run on ubuntu, so some knowledge about this platform may be useful. E.g. command line interaction. 

## Testing

We have performed unit tests with pytest. Pytest can be included in the pipeline directly, and you can choose which files to run with pytest. Recall that we need files with functions named

```python
def test_function(x):
    ...
    ...
    assert(computed==expected)
```

and then we can run 

```bash
pytest name_of_file.py
```

to execute the tests. 

## Building and deploying code

Often your code will run as an application in the cloud. The action of installing correct dependencies (build) for the application and update the most recent code to the application (deployment) is done in the actions file. We will not cover this part here. 

## Linting

We have touched upon coding conventions and styles. Lucky for us there is an automated tool for this to make sure coders that collaborate follows a certain standard. Tools will give you feedback on your code and often auto-correct when your code does not follow standard (linting) conventions. 

#### Black
Black is a popular code formatter for Python that enforces a consistent style for your code. Its primary goal is to automatically format Python code to make it more readable and maintainable, adhering to a consistent set of rules.

First, install `black` in your environment using pip:

```bash
pip install black
```
Now you can check whether your code will be formatted by
```bash
python -m black my_file.py --check
```
If we omit the "--check" black will automatically update my_file.py with needed changes. To run black (with changes) for all files in the current directory do:
```bash
python -m black .
```

#### Exercise
* Check whether your code adheres to black standards by installing and running black on one of your scripts
* Install a few other linting tools (e.g. flake8 and pylint). Run also these linting tools to check for differences. (flake8 my_file.py and pylint my_file.py to run)

# Routine tasks
Assume that we have a function that creates a set of outputs. This could be a report with a table, or trained parameters for a machine learning model. To get these values (outputs) to the application, the functions are most often run as a step in the automated pipeline. Let's say that we have a script:

In [ ]:

def f(x):
    return [2*x, x+5, 15*x]

The exact content of this function may change, and many developers have worked together (and separately) to update the function. This function is the basis for writing a report in table format. In the pipeline step we can for instance create a script that saves the results from running the function to the repository.

```python
from my_file import f
d0, d1, d2 = f(x)
with open("results_file.txt", "w") as file:
    content = f"{d0}, {d1}, {d2}"
    file.write(content)
```
We can save this file as update_results.py and in the pipeline we run:

```bash
python update_results.py
```

# The structure of the .yml-file

```yaml
name: Pylint

on: [push]

jobs:
  build:
    runs-on: ubuntu-latest
    strategy:
      matrix:
        python-version: ["3.8", "3.9", "3.10"]
    steps:
    - uses: actions/checkout@v4
    - name: Set up Python ${{ matrix.python-version }}
      uses: actions/setup-python@v3
      with:
        python-version: ${{ matrix.python-version }}
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install pylint
        pip install black
        pip install pytest
    - name: Run pytest
      run:  |
        pytest lectures/01-python-summary/lecture-scripts/test_*
    - name: Analysing the code with pylint and black
      run: |
        pylint $(git ls-files 'scripts/*.py')
        black scripts/*.py
```
        

## Exercise 

* Write a python script in the folder "scripts". You can copy one of your scripts from a weekly exercise. 

* Add, commit and push the changes to your forked repository.

* To run github actions on a forked repository you might need to go to "Actions" and enable actions on GitHub.

* Install pylint and black. Run pylint and black on the script that you added to the "scripts" folder. 

* Make necessary changes to the script such that pylint and black finishes successfully.

* Make changes to datestamp.txt on your local machine. Try to do git pull. Resolve the merge conflict
